### Google page rank
__MATH 420__ <br>
_Spring 2021_ <br>


We'll describe a method that is the basis of the Google page rank. Specifically, we would like a way to assign a numerical value, let's call it the _rank,_ that corresponds to the _popularity_ of a web page. 

To start our thinking about this, let's imagine that a popular webpage, say the _Wall Street Journal_ (WSJ), has a link to the _Kearney Hub._  The editor of the _Hub_ will be _thrilled_ with the traffic that might result from being linked from such a popular 
webpage. But if the _Kearney Hub_ links to the _Wall Street Journal,_ I'd guess that the editors of the WSJ would barely notice. So our first insight is that the rank of a page depends on the ranks of the pages that link to it. If a highly ranked page links to the _Kearney Hub,_ for example, it raises the rank of the _Hub._  But if a lowly ranked page links to the _Hub,_ it doesn't affect the rank of the _Hub_ all that much. 

Our second insight is that if a page links to many pages, that diminishes the influence of a link. A visitor to a page that links to a million other pages, for example, might click on any one of a million links, but a visitor to a page that only links to just ten pages has a better chance of visiting one of these ten pages. So the more links a page has, the less influence it has on the ranks of the pages it links to. We can think of each link from a web page as a vote, with the weight of each vote as $1/n$, where $n$ is the number of links from a web page. Thus, the sum of all the votes _from_ each web page is one. 

Given these insights, let's define the _rank_ of a page to equal to sum of the weights of the ranks that link to it. Again, the weight of each link is the reciprocal of the number of pages it links to. So if a page links to $10$ other pages, the weight of each link is $1/10$.

Let's take an example. Let's suppose we have four web pages labeled $A,B,C$, and $D$. 

Suppose pages $B$ and $C$ link to page $A$, and suppose page $B$ has a weight of $1/2$ (that is, it links to a total of two pages), and page $C$ has a weight of $1/3$ (thus page $C$ links to three pages). The rank of $A$ satisfies
$$
  \text{rank}(A) = \frac{1}{2}  \text{rank} (B) + \frac{1}{3}  \text{rank}(C).
$$

For the rank of $B$, suppose that pages $A$ and $C$ link to page $B$. And suppose the weight of page $A$ is $1/2$. Then
$$
  \text{rank}(B) = \frac{1}{2}  \text{rank} (A) + \frac{1}{3} \text{rank} (C).
$$
For the other two pages, let's suppose that 
$$
  \text{rank}(C) = \frac{1}{2}  \text{rank} (B) + \text{rank} (D) ,
$$
$$
\text{rank}(D) = \frac{1}{2}  \text{rank} (A) +  \frac{1}{3} \text{rank} (C). 
$$

In matrix notation, our equations are
$$
  \begin{bmatrix} A \\ B \\ C \\ D \end{bmatrix} = \begin{bmatrix} 0 & 1/2 & 1/3 & 0 \\ 1/2 & 0 & 1/3 & 0 \\ 0 & 1/2 & 0 & 1 \\ 1/2 & 0 & 1/3 & 0 \end{bmatrix} \begin{bmatrix} A \\ B \\ C \\ D \end{bmatrix}. 
$$
Here I tired of writing $\text{rank}(A)$, so I wrote $A$ instead; and similarly for the other variables.  We have expressed these linear equations in the form of a fixed point problem. Latter we'll use this fact as a way to find the ranks.

The coefficient matrix of our linear equation has several nice properties: 

(a) every column sum is one  <br>
(b) all entries are in the interval $[0,1]$. 

These properties are _not_ a coincidence, but they are a consequence of the way that the matrix was constructed. Any square matrix with these two properties is called a _Markov_ matrix (https://en.wikipedia.org/wiki/Stochastic_matrix). 

We won't go into the theory, but the Markov property of this matrix implies that the page rank corresponds to the probabilty that clicking on a randomly selected link will land user on a webpage.

Subtracting the left and right sides of these linear equations resuts in
$$
  \begin{bmatrix} 0 \\ 0 \\ 0 \\ 0 \end{bmatrix} = 
  \begin{bmatrix} -1 & 1/2 & 1/3 & 0 \\ 
                   1/2 & -1 & 1/3 & 0 \\ 
                   0 & 1/2 & -1 & 1 \\ 
                   1/2 & 0 & 1/3 & -1 \end{bmatrix} \begin{bmatrix} A \\ B \\ C \\ D \end{bmatrix}. 
$$
The equations for the unknowns $A, B, C$, and $D$ are homogeneous and linear. Accordingly, any scalar multiple of a solution provides another solution. This freedom allows us to require that the sum of the ranks have a specific value. Requiring that sum of the ranks be one,
our linear equations are
$$
  \begin{bmatrix} 0 \\ 0 \\ 0 \\ 0 \\ 1\end{bmatrix} = 
  \begin{bmatrix} -1 & 1/2 & 1/3 & 0 \\ 
                   1/2 & -1 & 1/3 & 0 \\ 
                   0 & 1/2 & -1 & 1 \\ 
                   1/2 & 0 & 1/3 & -1 \\
                   1 & 1 & 1 & 1  \\
                   \end{bmatrix} \begin{bmatrix} A \\ B \\ C \\ D \end{bmatrix}. 
$$
These equations are _not_ homogeneous and they are _over determined_ (the number of unknowns is greater than the number of knowns).  It is _not_ certain that the equations have a solution, and if they do have a solution, it's possible that some of the ranks will be negative. Following the logic of how these equations were determined, a negative rank doesn't make much sense.

The Perron-Forbenious theorem comes to the rescue. This theorem tells us that for any Markov matrix $M$, there is a vector $\mathbf{x}$ such that $M \mathbf{x} = \mathbf{x}$
and $1 = \sum x_k$, where each $x_k$ is nonnegative. But the solution isn't always unique.

More generally, if $\mathbf{x}$ is a nonzero vector and $\lambda$ is a number such that $M \mathbf{x} = \lambda \mathbf{x}$, we say that $\mathbf{x}$ is an _eigenvector_ of the matrix $M$ with _eigenvalue_ $\lambda$. In eigenvector/eigenvalue language, the Perron-Forbenious theorem tells us that if $M$ is a Markov matrix, the matrix $M$ has at least one eigvector with eigenvalue one.


Let's have Julia solve the linear equations. To do this, we'll need the package `LinearAlgebra`. 

In [1]:
using LinearAlgebra

We define the matrix `M` and the right-hand side `b` by hand. After that, we can 
use Julia's `\` operator to solve the over determined linear system.

In [2]:
M = [-1 1/2 1/3 0; 1/2 -1 1/3 0; 0 1/2 -1 1; 1/2 0 1/3 -1; 1 1 1 1];

In [3]:
b = [0 ; 0 ; 0 ; 0 ; 1];

As the theory tells us, every solution is nonnegative and the sum of the solutions is one.

In [4]:
ranks = M \ b

4-element Vector{Float64}:
 0.22222222222222204
 0.22222222222222224
 0.3333333333333334
 0.22222222222222227

In [5]:
sum(ranks)

1.0

Here is a function `PR` that does these steps for us.

In [6]:
function PR(M::AbstractMatrix{T}) where T
    n, m = size(M)
    if n != m
        throw(ArgumentError("The argument to `PR` must be square matrix."))
    end
    #  Subtract the identity matrix and append a row of ones
    M = vcat(M - I, ones(T, 1, n))    
    # Create the right-hand side vector b with zeros and set the last element to one
    b = zeros(T, n+1)
    b[end] = one(T)
    # Solve the linear system M x = b and return the solution
    M \ b
end


PR (generic function with 1 method)

Let's apply `PR` to the problem we solved by hand. The coefficient matrix is

In [7]:
M = [0 1/2 1/3 0; 1/2 0 1/3 0; 0 1/2 0 1; 1/2 0 1/3 0];

This agrees with our previous result:

In [8]:
PR(M)

4-element Vector{Float64}:
 0.22222222222222204
 0.22222222222222224
 0.3333333333333334
 0.22222222222222227

Returning to the idea that the ranks are the solution to a fixed point problem, an alterative way to solve the equations is to use fixed point iteration. When the input matrix is a Markov matrix and sum the members of the the initial term of the fixed 
sequence is one, this function will return a solution whose terms sum to one.

Here is a simple function that does the iteration. 

In [9]:
function fixed_point(M, x0, tol, iter=0)
    if iter  < 100
       x1 = M * x0
       if norm(x1-x0, Inf) < tol x1 else fixed_point(M, x1, tol,iter+1) end
    else
        println("Fixed point sequence doesn't seem to converge.")
        nothing
    end
end

fixed_point (generic function with 2 methods)

Let's try it--we'll try an initial point of $[1,0,0,0]$.

In [10]:
M = [0 1/2 1/3 0; 1/2 0 1/3 0; 0 1/2 0 1; 1/2 0 1/3 0];

In [11]:
fixed_point(M, [1; 0 ; 0; 0], 1.0e-6)

4-element Vector{Float64}:
 0.22222228348255157
 0.22222229093313217
 0.3333331346511841
 0.22222229093313217

And let us try a different starting value. We get the same fixed point.

In [12]:
fixed_point(M, [1/4; 1/4 ; 1/4; 1/4], 1.0e-6)

4-element Vector{Float64}:
 0.22222232818603516
 0.22222232818603516
 0.33333301544189453
 0.22222232818603516

These numbers are familiar! This is exactly the result we got by solving the
linear equations using the `\` operator.  For our matrix, we can show that if the sum of the members of `x` is one, the sum of the members of $M x$ is also one. Since started with a vector whose sum of components was one, the method returns a vector that also has a sum of components of one.

We've described what Wikipedia (https://en.wikipedia.org/wiki/PageRank#Simplified_algorithm) refers to the _simplified version_.  

The Patent for the Google Page Rank (https://patentimages.storage.googleapis.com/db/8f/cb/dad63e985797ec/US7058628.pdf) replaces the Markov matrix $M$ for the simplified version by 
$$
  \frac{\alpha}{N} I  + (1 - \alpha) M,
$$
where $N$ is the number of nodes, $I$ is an identity matrix, and $\alpha \in [0,1]$ is called the _damping factor_. In general, this is _not_ a Markov matrix, and its largest eigenvalue (called the _dominant eigenvalue_) is strictly less than one. Actually, since all eigenvalues are inside the unit circle, it can be shown that _every_ fixed point sequence for the matrix $  \frac{\alpha}{N} I  + (1 - \alpha) M$ converges to the zero vector. So how does this method work? Buried in the Patent application is 

"_Note that in order to ensure convergence, the norm of p, must be made equal to 1  after each iteration_"

And this means that the original method modifies the fixed point sequence by dividing each term fixed point sequence by a norm (which norm, the one, two, or infinity, doesn't matter). This is known as the power method for finding the dominant eigenvalue (see https://en.wikipedia.org/wiki/Power_iteration).

With or without a damping factor, the matrix used can have two or more linearly independent eigenvectors corresponding to the eigenvalue with the greatest magnitude. This happens, for example, when there are two or more nonempty disjoint sets of web pages (call them clusters) that are linked to other members of the subset, but not other clusters. Here is an example

In [13]:
M = [0 1 0 0; 1 0 0 0; 0 0 0 1; 0 0 1 0]

4×4 Matrix{Int64}:
 0  1  0  0
 1  0  0  0
 0  0  0  1
 0  0  1  0

There are two column vectors $x$ with nonnegative members that are solutions to the linear system $M x = x$. These solutions are

In [14]:
sol1 = [1/2;1/2;0;0];

In [15]:
M  * sol1 -  sol1

4-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0

In [16]:
sol2 = [0;0;1/2;1/2];

In [17]:
M  * sol2 -  sol2

4-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0

Every linear combination of these solutions is a solution; so another solution is

In [18]:
sol3 = (sol1 + sol2)/2

4-element Vector{Float64}:
 0.25
 0.25
 0.25
 0.25

In [19]:
M*sol3 - sol3

4-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0

But our linear equation method given by the function `PR` gives a four-way tie for first place. This corresponds to the solution `sol3`.

In [20]:
PR(M)

4-element Vector{Float64}:
 0.25
 0.2500000000000002
 0.25000000000000006
 0.25000000000000006

How does Julia's `\` operator chose one solution from infinitely many solutions? The user documentaiton for `\` says that the solution is the 

_minimum-norm least squares solution computed by a pivoted QR factorization of A and a rank estimate of A based on the R factor_.

The meaning of this is beyond our class.

But the fixed point mathod gives us something else. When the starting value is 
$[1,0,0,0]$, we get an error because the fixed point sequence doesn't converge.

In [21]:
fixed_point(M, [1; 0 ; 0; 0], 1.0e-2)

Fixed point sequence doesn't seem to converge.


But changing to a starting value of $[1/4,1/4,1/4, 1/4]$, the fixed point sequence converges.

In [22]:
fixed_point(M, [0.25; 0.25 ; 0.25; 0.25], 1.0e-2)

4-element Vector{Float64}:
 0.25
 0.25
 0.25
 0.25

One way insure that the solution is unique is to introduce a fictitious ''super node'' that is linked to every page and every page is linked to the super node. Effectively, the super node idea includes the possibility that a user will visit a page by entering a url instead of randomly clicking.

Google ranks, I suppose, hundreds of billions (maybe trillions?) or so of pages. Solving the linear equations for the rank using row reduction for such a huge matrix isn't, I think, possible. And it isn't needed either. The iterative process can be done quickly to find the page rank. Reasonable estimates are that it takes Google a few weeks to construct the graph of links and a few days to compute the page ranks.

The Google Page Rank is named partially in honor of Larry _Page,_ one of the co-founders of Google, not as you might guess after a web _page._ But the idea of using eigenvalues to rank options was popularized by the mathematician Thomas Saaty _decades_ before Google used it to rank pages. Stigler's law of eponymy, says that "states that no scientific discovery is named after its original discoverer." (https://en.wikipedia.org/wiki/Stigler%27s_law_of_eponymy). And so it is with the Google Page Rank. I'd say that the Patent Office _flubbed_ by issuing a patent for the Google Page Rank.

The history of the concept of an eigenvalue goes back to at least Euler (1707 – 1783). About 230 years after Euler used eigenvectors and eigenvalues to describe the motion of ridged bodies, Larry Page used the same concept to launch one of the largest companies of all time.

For more information, see https://en.wikipedia.org/wiki/PageRank ; and see https://en.wikipedia.org/wiki/Thomas_L._Saaty

Here is an example that shows that using the Google Page Rank damping factor along with fixed point iteration makes the fixed point sequence converge to the zero vector.

In [23]:
alpha = 0.15;

In [24]:
N = 4;

With a nonzero damping factor, the coefficient matrix is _not_ Markov. Since the modified matrix is not a Markov matrix, the probablistic interpertation of the matrix is _invalid_. Confusingly, the patent application uses probabilty language for method with the damping factor. Arguably, I'd say the patent application is a more of a bussines plan then it is  science. 

In [25]:
xx = alpha/N * I + (1-alpha) * M

4×4 Matrix{Float64}:
 0.0375  0.85    0.0     0.0
 0.85    0.0375  0.0     0.0
 0.0     0.0     0.0375  0.85
 0.0     0.0     0.85    0.0375

In [26]:
fixed_point(xx, [1; 0 ; 0; 0], 1.0e-6)

4-element Vector{Float64}:
 7.559789073422259e-6
 7.563896533750736e-6
 0.0
 0.0

But remember that the patent application says that the norm of the vector must be adjusted to one after each iteration. Assuming this means dividing by the one norm after each iteration, a modified function for the fixed point sequence is


In [27]:
function fixed_point(M, x0, tol, iter=0)
    if iter  < 1000
       x1 = M * x0
       x1 = x1/norm(x1,1)
       if norm(x1-x0, Inf) < tol x1 else fixed_point(M, x1, tol,iter+1) end
    else
        println("Fixed point sequence doesn't seem to converge.")
        nothing
    end
end

fixed_point (generic function with 2 methods)

The use of the damping factor does _not_ create a unique fixed point.

In [28]:
fixed_point(xx, [1; 0 ; 0; 0], 1.0e-5)

4-element Vector{Float64}:
 0.49999526002509537
 0.5000047399749047
 0.0
 0.0

In [29]:
fixed_point(xx, [0; 0 ; 1; 0], 1.0e-5)

4-element Vector{Float64}:
 0.0
 0.0
 0.49999526002509537
 0.5000047399749047

Google's patent application says that in addition to the damping factor, it uses an intial 
term of the fixed point sequence with each member with the same value. I'm not sure of the theory behind the method, but apparently, with these modifications, the fixed point sequence converges to a unique soljution. My guess is that the damping factor has more to do with preventing overflow than with convergence. 